In [456]:
%matplotlib inline

In [457]:
import hvplot.pandas

In [458]:
import pandas as pd
import numpy as np
from fastcore import *
from fastcore.all import * 

In [459]:
base_path = Path('/mnt/c/trial/results')

In [460]:
experiment_paths = base_path.ls()
experiment_paths

(#3) [Path('/mnt/c/trial/results/matan-lr-download'),Path('/mnt/c/trial/results/new-minisat-localrestart'),Path('/mnt/c/trial/results/original')]

In [462]:
# import os
# for f_p in Path('/mnt/c/trial/results/matan-lr-download').ls():
#     f_name = str(f_p)
#     os.rename(f_name, f_name.replace('Task-',''))

In [463]:
new_names = [p.name for p in Path('/mnt/c/trial/results/new-minisat-localrestart').ls()]
original = [p.name for p in Path('/mnt/c/trial/results/original').ls()]

len(set(original).intersection(set(new_names)))

397

In [464]:
def read_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.read().splitlines()
        
    d = {}
    for l in lines:
        value = l.split(':')
        d[value[0]] = value[1]
    
    return d

In [465]:
def read_experiment_results(name):
    exp = base_path / name
    df = pd.DataFrame([read_file(file_name) for file_name in exp.ls() if file_name.name in names])
    df['experiment'] = name
    return df

In [466]:
a = experiments[0]
experiment_names = [p.name for p in experiment_paths]
experiment_names

['matan-lr-download', 'new-minisat-localrestart', 'original']

In [467]:
df = pd.concat([read_experiment_results(name) for name in experiment_names], axis=0)

In [468]:
df['num_backtracks'] = df['num_backtracks'].fillna('0')
df['num_restarts'] = df['num_restarts'].fillna('0')
df['vdh'] = df['vdh'].fillna('MiniSAT')
df['solver'] = df['solver'].fillna('edusat')
df.loc[df['experiment']=='matan-lr-download','vdh']='LR'

In [469]:
df = df.astype({'nclauses': int, 'num_assignments': int, 'num_backtracks': int, 'num_decisions': int, 'num_learned': int, 
                    'nvars': int, 'runtime': float})

In [470]:
df.shape

(1191, 15)

In [471]:
df.head(3).T

,0,1,2
filename,3col200_5_3.shuffled.cnf,3col200_5_5.shuffled.cnf,3col20_5_10.shuffled.cnf
nclauses,1706,1706,176
num_assignments,10085237,5061042,258
num_backtracks,69313,39704,29
num_decisions,628263,278273,50
num_learned,69313,39704,29
num_restarts,31398,12870,2
nvars,400,400,40
runtime,293.721,105.236,0.0
solver,new_solver,new_solver,new_solver


# Check for errors

In [478]:
df.groupby(['experiment', 'error']).agg({'experiment':'count'})

experiment
experiment               error                                           
matan-lr-download        Found conflict at level dl=0                 122
                         Timeout of **1200** seconds exceeded          19
new-minisat-localrestart Found conflict at level dl=0                 125
                         Timeout of **1200** seconds exceeded           4

Great! No runtime errors! Only conflicts and timeouts!

# Statistics per experiment

In [474]:
df.groupby(['experiment', 'solver', 'vdh', 'status']).agg({'experiment': 'count', 'runtime': 'mean',
                'num_assignments': 'mean', 'num_learned': 'mean', 'num_backtracks': 'mean', 'nclauses': 'mean'}).\
                    rename({'experiment': 'count'}, axis=1)

count      runtime  \
experiment               solver     vdh     status                        
matan-lr-download        new_solver LR      ERROR       19  1200.014579   
                                            SAT        256    30.627813   
                                            UNSAT      122    26.521754   
new-minisat-localrestart new_solver MiniSAT ERROR        4  1200.007250   
                                            SAT        268    19.028612   
                                            UNSAT      125    34.156104   
original                 edusat     MiniSAT SAT        259    37.457591   
                                            TIMEOUT     19  1200.064737   
                                            UNSAT      119    26.359538   

                                                     num_assignments  \
experiment               solver     vdh     status                     
matan-lr-download        new_solver LR      ERROR       3.940188e+07   
                                            SAT         1.520364e+06   
                                            UNSAT       2.220856e+06   
new-minisat-localrestart new_solver MiniSAT ERROR       2.346612e+07   
                                            SAT         2.150957e+06   
                                            UNSAT       3.132378e+06   
original                 edusat     MiniSAT SAT         8.866287e+10   
                                            TIMEOUT     2.131959e+12   
                                            UNSAT       3.708911e+10   

                                                       num_learned  \
experiment               solver     vdh     status                   
matan-lr-download        new_solver LR      ERROR    178801.052632   
                                            SAT        5742.269531   
                                            UNSAT      6394.057377   
new-minisat-localrestart new_solver MiniSAT ERROR    114522.000000   
                                            SAT        5220.902985   
                                            UNSAT      8998.680000   
original                 edusat     MiniSAT SAT        3311.942085   
                                            TIMEOUT   62371.894737   
                                            UNSAT      5132.865546   

                                                     num_backtracks  \
experiment               solver     vdh     status                    
matan-lr-download        new_solver LR      ERROR     178801.052632   
                                            SAT         5742.269531   
                                            UNSAT       6394.057377   
new-minisat-localrestart new_solver MiniSAT ERROR     114522.000000   
                                            SAT         5220.902985   
                                            UNSAT       8998.680000   
original                 edusat     MiniSAT SAT            0.000000   
                                            TIMEOUT        0.000000   
                                            UNSAT          0.000000   

                                                        nclauses  
experiment               solver     vdh     status                
matan-lr-download        new_solver LR      ERROR    5677.210526  
                                            SAT      4768.175781  
                                            UNSAT    4274.204918  
new-minisat-localrestart new_solver MiniSAT ERROR    5145.250000  
                                            SAT      4796.608209  
                                            UNSAT    4351.208000  
original                 edusat     MiniSAT SAT      4732.131274  
                                            TIMEOUT  7493.736842  
                                            UNSAT    4050.168067

In [475]:
df.hvplot.scatter(x='nclauses', y='runtime', by=['status']).options(title='Runtime by #clauses')

:NdOverlay   [status]
   :Scatter   [nclauses]   (runtime)

In [507]:
df[df['filename']=='bart22.shuffled.cnf']

,filename,nclauses,num_assignments,num_backtracks,num_decisions,num_learned,num_restarts,nvars,runtime,solver,status,timeout,vdh,error,experiment
104,bart22.shuffled.cnf,1596,30554930,166250,3630644,166250,152086,288,1200.013,new_solver,ERROR,1200,LR,Timeout of **1200** seconds exceeded,matan-lr-download
104,bart22.shuffled.cnf,1596,668782,9690,47813,9690,2194,288,6.109,new_solver,SAT,1200,MiniSAT,NaN,new-minisat-localrestart


In [506]:
df.loc[lambda x: x['experiment'] != 'original'].sort_values(['runtime'], ascending=False)[['filename', 'vdh','experiment', 'runtime', 'status']].head(30)

,filename,vdh,experiment,runtime,status
105,bart23.shuffled.cnf,LR,matan-lr-download,1200.066,ERROR
100,bart18.shuffled.cnf,LR,matan-lr-download,1200.027,ERROR
171,ezfact48_10.shuffled.cnf,LR,matan-lr-download,1200.022,ERROR
109,bart27.shuffled.cnf,LR,matan-lr-download,1200.021,ERROR
128,cnt08.shuffled.cnf,LR,matan-lr-download,1200.016,ERROR
146,dp12s12.shuffled.cnf,LR,matan-lr-download,1200.015,ERROR
110,bart28.shuffled.cnf,LR,matan-lr-download,1200.015,ERROR
182,hgen5-v125-s821831669.cnf,LR,matan-lr-download,1200.014,ERROR
104,bart22.shuffled.cnf,LR,matan-lr-download,1200.013,ERROR
112,bart30.shuffled.cnf,LR,matan-lr-download,1200.012,ERROR
